# Call the 2 algoritms Hungarian and Auction Vickrey

### Work for University Class

MRobalinho

30-11-2020

In [7]:
import os
import numpy  as np
import pandas as pd
import random
import datetime

In [3]:
'''
# pip install Munkres
Munkres implementation for Python
Let C be an n by n matrix representing the costs of each of n workers to perform any of n jobs.
The assignment problem is to assign jobs to workers in a way that minimizes the total cost. 
Since each worker can perform only one job and each job can be assigned to only one worker 
the assignments represent an independent set of the matrix C.

One way to generate the optimal set is to create all permutations of the indexes necessary 
to traverse the matrix so that no row and column are used more than once.

http://software.clapper.org/munkres/

22-11-2020

Non-square Cost Matrices
The Munkres algorithm assumes that the cost matrix is square. However, it’s possible to use
a rectangular matrix if you first pad it with 0 values to make it square. 
This module automatically pads rectangular cost matrices to make them square.

Notes:

The module operates on a copy of the caller’s matrix, so any padding will not be
seen by the caller. The cost matrix must be rectangular or square. 
An irregular matrix will not work.
'''

from munkres import Munkres , print_matrix
import numpy as np
import pandas as pd
#--------------------------------

def hungarian(h_matrix):
    print('-------------------------------------')
    print('|------ Hungarian Algorithm --------|')
    print('-------------------------------------')

    arr = h_matrix
   
    # Number rows and columns
    l, c = h_matrix.shape
    x_agents = l
    x_tasks  = c
    
    if l > c:
        width = l
    else:
        width = c
        
    print('Original Matrix - Agents (row):',x_agents,'  Tasks (col):',x_tasks) 

    #------------------------
    # create a list with Agents
    i =0
    agents = []
    while i < l:
        name = 'Agent_'+str(i)
        agents.insert(i,name)
        i = i + 1
    #print(agents)   

    #------------------------------------
    # Obtain Max an min value from matrix
    matrix_arr = np.array(h_matrix)
    
    list_matrix = matrix_arr.tolist()
    
    #print(list_matrix, type(list_matrix))
    
    # Max and Minimum
    list_matrix = matrix_arr.flatten()
    #max_value = max([max(l) for l in list_matrix])
    #min_value = min([min(l) for l in list_matrix])
    
    max_value, min_value = max_and_min(matrix_arr)
   
    print('Max Value:', max_value,'  Min Value:', min_value)

    #-----------------------------------------
    # Transform Non Square to Square
    nonsquare = arr
    square = []
    if ( x_agents < x_tasks ):  # Only transform square if we have more tasks than agents 
        print('-> Transform to Square Matrix with:', width ,' by ', width)
        square = np.zeros((width, width))
        square = np.array(square)                 # Transform Matrix in numpy array

        np.place(square, square == 0 , max_value) # Replace Virtual lines with Max Matrix

        square[:l, :c] = nonsquare                # concatenate both array square   and non square
    else:
        square = arr

    #square
    #------------------------------------------
    # Assumes the agent price to the tasks, for each dummy agent
    i   = 0
    n_i = 0
    n_l = l    
    while i < (c - l ):
        if n_i > (l - 1):  # Cycle to reset the agent to the first to new allocation
            n_i = 0 
        print('-> Clone Agent:', n_i ,' to ', n_l)
        square[n_l] = nonsquare[n_i]
        n_l += 1 
        n_i += 1
        i   += 1
    #square
    #------------------------------------------
    # Insert agents to new lines
    i       = 0
    n_agent = l
    while i < (c - l ):
        name = agents[i]
        agents.insert(n_agent, name)
        n_agent = n_agent + 1 
        i = i + 1
    agents
    #--------------------------------------
    # Transform numpy array to a list
    square_list = np.array(square).tolist()
    square_list

    # - inicialize Algoritm Munkres
    m = Munkres()

    #-------------------------------------
    indexes = m.compute(square_list)

    print_matrix(square_list, msg='Lowest cost through this square  matrix:')
    total = 0
    print('--------------')
    print('Assigned Tasks Square Matrix')
    for row, column in indexes:
        value = square_list[row][column]
        total += value
        print(agents[row], {row}, ' Assigned task:', {column}, '-> Cost:', {value})
    print(f'Total cost Square: {total}')
    x_alg = 'Hungarian'
    x_agent = l
    x_task = c
    x_total = total
    indexes

    #---------------------------------------------------------------
    if l != c :
        print('------------------------------------------------------')
        non_square_list = np.array(h_matrix).tolist()
        indexes = m.compute(non_square_list)

        print_matrix(non_square_list, msg='Lowest cost through this non square matrix:')
        total = 0
        print('--------------')
        print('Assigned Tasks Non Square Matrix')
        for row, column in indexes:
            value = non_square_list[row][column]
            total += value
            print(agents[row], {row}, ' Assigned task:', {column}, '-> Cost:', {value})
        print(f'Total cost Non Square: {total}')
        
        if ( x_agents > x_tasks ):
            x_alg = 'Hungarian_NSQ'
            x_agent = l
            x_task = c
            x_total = total
        
    return x_alg, x_agent, x_task, x_total

In [4]:
'''
Vickrey Auction

Vickrey auctions are a type of sealed-bid auction where all participants bid for the lot 
at the same time without knowing how their competitors are bidding. As a result, 
the winner of the auction is the bidder with the highest bid. 
However, the winner does not pay the highest bid, but the second-highest bid amount. 
This type of auction encourages the participants to bid exactly how much they value 
the good being sold.

https://corporatefinanceinstitute.com/resources/knowledge/other/vickrey-auction/

http://www.masfoundations.org/mas.pdf

MRobalinho : 4-12-2020
'''

import numpy as np
#  Main start for auction vickrey
def main_auction_vickrey(matrix_arr, x_tasks, x_agents):
    print('-------------------------------------')
    print('|-- Vickrey Auction Algotithm ------|')
    print('-------------------------------------')
    
    # Number Agents and task
    print('Original values have Agents:', x_agents, '   Tasks:', x_tasks)
    print('    >Original Matrix:')
    print('    ', matrix_arr)
    
    # Transpose original Matrix
    matrix_t = matrix_arr.transpose()
    
    #matrix_t
    #print('    >Transposed Matrix:')
    #print('    ', matrix_t)

    arr = np.array(matrix_t)
    # Number rows and columns
    l, c = arr.shape
    
    # Create array with agent score
    score_x = []
    row, col  = 1, x_agents
    score_x = [[0 for x in range(col)] for y in range(row)] 
    score_arr = np.array(score_x)

    # Maximum and minimum value
    max_value, min_value = max_and_min(matrix_arr)
    print('After transpose: Rows:',l, 'Columns:', c, '  Max:', max_value,'  Min:', min_value)

    # Creates a null list containing l(one for each task) lists, 
    # each of 4 items(4 informations), all set to 0
    # rows = tasks
    # columns (0=Task, 1=agent, 2 = bid winner)
    row, col  = l, 3;
    bk_task   = row  # backup for task number
    bk_agents = c    # Backup for agent number
    
    winners = [[0 for x in range(col)] for y in range(row)] 
    winners_arr = np.array(winners)

    # Calculate Bid Winners
    winners_arr = calc_bid(matrix_t, winners_arr, score_arr)
     
    # Normalization - Some agents with many task and others with none
    verify_normalization(matrix_t, score_arr, winners_arr)
            
    #  call last scores to print
    end_run = 1
    create_scores(winners_arr, x_agents, bk_task, score_arr, end_run)
    
    # Calculate total process value
    x_total = calc_total(winners_arr)
    
    return x_total

#-----------------------------------------------------------------
# Calculate the best bids for each task
def calc_bid(matrix_t, winners_arr, score_arr):
    
    # Select maximum bid in each task (each row)
    arr = np.array(matrix_t)
    
    # Number rows and columns
    l, c = arr.shape
    x_agents = c
    x_task = l
   
    # Create the winners array
    n_task = 0
    for task in matrix_t:
        n_agent = 0
        for agent in range(len(task)):
            if n_agent == 0:
                winners_arr[n_task,0] = n_task # put Task number in col 0
            n_agent += 1
        n_task += 1
    print('Calc Bid generated winner matrix:', winners_arr)
    
    # Calculate winners
    n_task = 0
    for task in matrix_t:
        max_bid = 99999   # Change to hight (99999) or Low (0)
        print('Calc winners - Task number:',n_task)
        n_agent = 0
        for agent in range(len(task)):
            
            this_bid = task[n_agent]  # Bid Value from row/column matriz              
                    
            # Select Best bid
            win = False
            if this_bid < max_bid: # Change to hight (99999) or Low (0)
                older_max_bid = max_bid
                max_bid = task[n_agent]
                #print('    Win Best bid, agent ->', n_agent, '  Bid:', this_bid, ' Older Max Bid',older_max_bid)
                winners_arr[n_task,0] = n_task
                winners_arr[n_task,1] = n_agent
                winners_arr[n_task,2] = max_bid
                win = True
                
            # If  a Tie we consider the best bidder agent    
            if this_bid == max_bid and win == False:    
                                
                # Create score to obtain the best winner agent
                score_arr = create_scores(winners_arr, x_agents, x_task, score_arr, 0)
                print('    Tie agent:',n_agent, 'Task:',task[n_agent], 'Max Bid:', max_bid, ' Scores:', score_arr)
            
                ix = 0                
                # Find last winner for the same task
                if score_arr[0, n_agent] == 0: # If Agent has no task he win
                    last_bid_agent = n_agent
                    print('    ---> Best bid tie, first bid, agent ->', last_bid_agent, '  Bid:', max_bid)
                else:
                    # Find the Agent that wins more times
                    last_bid_agent = 0
                    for ix in range(len(task)):            
                        if max_bid == task[ix] and ix < n_agent:               
                            last_bid_agent = ix
                            print('    --> Calc Tie Score This:', score_arr[0, n_agent],'  Older',score_arr[0, ix]) 
                        ix += 1
                    print('    ---> Best bid tie, agent ->', last_bid_agent, '  Bid:', max_bid)
                
                winners_arr[n_task,0] = n_task
                winners_arr[n_task,1] = last_bid_agent
                winners_arr[n_task,2] = max_bid    
            n_agent += 1
        n_task += 1

    # print array winners    
    print('Winners:')
    #winners_arr
     
    return winners_arr

#----------------------------------------------------------------------
def verify_normalization(matrix_t, score_arr, winners_arr):
           
    # Normalization - Some agents with many task and others with none
    try_normalization = False
    t_task, t_agents  = matrix_t.shape 
    
    # generate scores
    score_arr = create_scores(winners_arr, t_agents, t_task, score_arr, 0)
    
    # zero scores for recalculation
    i = 0
    while i < t_agents:
        if score_arr[0, i] == 0:   # Has Agents without tasks
            try_normalization = True
        i += 1
             
    #--------------------
    xi = 1
    if try_normalization == True:
        print('-> Start Normalization, with :', t_task, 'tasks,  and :', t_agents,' Agents')
        print('Normalization -> scores :', score_arr)
        print('              -> winners:', winners_arr)
    else:
        print('-> Dont need normalization:', score_arr)
        
    while try_normalization == True:
        # generate scores
        score_arr = create_scores(winners_arr, t_agents, t_task, score_arr, 0)
              
        n_agent_zero  = 999999
        n_agent_multi = 999999
              
        i = 0
        while i < t_agents:
            if score_arr[0, i] == 0:   # Has Agents without tasks
                n_agent_zero = i
            if score_arr[0, i] > 1:  # Search Agents with many tasks, need Normalization
                n_agent_multi = i 
            i += 1              
 
        #print('Verify normalization, cicle:', xi,', zero agent:', n_agent_zero, ' Multi agent:', n_agent_multi )    
        if n_agent_zero != 999999 and n_agent_multi != 999999: # Has some with wulti task and some without task   
            do_normalization(matrix_t, winners_arr, n_agent_zero, n_agent_multi)
        else:
            try_normalization = False
            print('-> Finish Normalization')
        xi += 1
        
    return winners_arr, score_arr

#------------------------------------------------------
def do_normalization(matrix_t, winners_arr, agent_zero, agent_multi):
    print('--- Do Normalization Vickrey auction----')
    #print(winners_arr)
    
    t_task, t_agents  = matrix_t.shape
    n_task = 0
    n_do   = True
    for task in range(len(winners_arr)):
        max_bid = 99999   # Change to hight (99999) or Low (0)
        
        n_agent = winners_arr[n_task,1]
        #print('>> Normalization - Task number:',n_task, 'Agent:', n_agent, 'Agent zero:',agent_zero,' Agent multi:', agent_multi )
        if n_agent == agent_multi and n_do == True and n_agent <= t_agents:
            new_bid = matrix_t[n_task, agent_zero]
            winners_arr[n_task,0] = n_task        # Dont' change the task, assign to another agent
            winners_arr[n_task,1] = agent_zero    # assign to an agent with zero task
            winners_arr[n_task,2] = new_bid       # Assign Bid from agent zero
            #print('-->> Normalization - Task number:',n_task, 'Agent zero:',agent_zero,' New Bid:', new_bid )
            n_do = False  # do only one time
        n_task += 1
    return winners_arr

#------------------------------------------------------
def calc_total(winners_arr):
    print('--- Calculate totals Vickrey auction----')
 
    x_task  = 0
    x_total = 0
    for task in winners_arr:
        x_total  += winners_arr[x_task,2]
        x_task += 1
    
    xt = x_total
    print('Bid Total value:',x_total)
    print('Winners Matrix row=Tasks , Col_0 =Task number, Col_1=Agent, Col_2=Bid Value')
    winners_arr
          
    return x_total

#---------------------------------------------------------------------
# Create the agent scores
def create_scores(winners_arr, x_agents, bk_task, score_arr, end_run):
    
    t_task   = bk_task
    t_agents = x_agents
  
    # Transpose winners array
    winners_arr_t = winners_arr.transpose()
    if end_run == 1:
        print('Initial Create scores_1:', x_agents, score_arr) 
        print(winners_arr_t)     
    
    # zero scores for recalculation
    i = 0
    while i < x_agents:
        score_arr[0, i] = 0
        i += 1
   
    # Select maximum bid in each task (each row)
    n_row = 0
    # Row 1 has the winners agents, for each task
    if end_run == 1:
        print('-------------------------')
        print('WINNERS :')
 
    for row in winners_arr_t:
        if n_row == 0 :  # row with winners          
            i = 0    
            for i in range(len(row)):     
                #print(' > Tasks number:',winners_arr_t[0, i], ' Winner:', winners_arr_t[1, i], '=', winners_arr_t[2, i])

                if winners_arr_t[2, i] >= 0: # Bid value need to be > 0:
                    winner_agent = winners_arr_t[1, i]  # Agent number
                    if winners_arr_t[2, i] > 0:
                        score_arr[0, winner_agent] += 1
                    if end_run == 1:
                        print('  Task > ', i, ' Agent:', winner_agent, '  Bid:', winners_arr_t[2, i])
                i += 1
        n_row += 1
        
    # Score pontuation to the agents winners
    if end_run == 1:
        print('Agent scores:',score_arr)
    return(score_arr)


#----------------------------------------------------------
# Create Bid matriz usin original matriz and insert a % price variation
def create_bid_matrix(b_matrix, par_var):
    
   # par_var = 50  # % price variation for bid
    var_p = par_var / 100
    print(' > Create Bid Matrix :', par_var , '%')
    #b_matrix
    
    bid_matrix = b_matrix
    line = 0
    col = 0 
    for row in bid_matrix: 
        for col in range(len(row)):  
            min_value = int(row[col] - (row[col] * var_p))
            max_value = int(row[col] + (row[col] * var_p))

            #print(min_value, max_value)
            rand_value = random.randint(min_value, max_value)
            if rand_value == 0:
                rand_value = 1    
            #print('  Row >', line, col, 'Act.Value:', row[col], '  Min:', min_value, ' Max:',max_value, 'Random:',rand_value, )
            row[col] = rand_value
            col += 1
        line += 1
    print('Bid matrix:',bid_matrix)
    
    return bid_matrix
#------------------------------------------------------------
# Define Maximum and Mininum vales
def max_and_min(z_matrix):
    line = 0
    col = 0 
    min_value = 999999999999999
    max_value = 0
    
    for row in z_matrix: 
        for col in range(len(row)):  
            if row[col]  < min_value:
                min_value = row[col]
            if row[col]  > max_value:
                max_value = row[col]
            col += 1
        line += 1
    return max_value, min_value

In [10]:
def global_call(x_agent, x_task, k_matrix, x_count):
    
    matrix = k_matrix
    arr    = np.array(matrix)
    
    print('============= Hungarian algorithm (', x_count,') ======================')
    # Call Hungarian algorithm
    x_alg, x_agent, x_task, x_total = hungarian(arr)

    # Save values
    df_h = pd.DataFrame([([x_alg, x_agent, x_task, x_total ])], columns=['Algoritmo','Agents', 'Tasks', 'Cost'])

    arr = np.array(matrix)
    # Number rows and columns
    l, c = arr.shape

    #------------------------------------------------------------------
    # Call Auction algorithm Vickrey

    # bid_number = num. licitaçoes
    # agent_number = numero agentes
    # max_value_bid = maximum value for bid

    bid_number = 1
    agent_number = l  # Number agents from the matrix
    #max_value_bid = par_max_value
    tasks = c         # Number of tasks from the matrix

    print('============= Vickrey Auction algorithm (', x_count,') ======================')

    # Transpose
    matrix_arr = np.array(k_matrix)
    matrix_t = matrix_arr.transpose()
    
    # Create Bid Matrix using original matrix (matrix, % bid price variation)
    b_matrix = matrix_arr
    #b_matrix = create_bid_matrix(matrix_arr, par_var)
    
    # Call auction algorithm
    x_total = main_auction_vickrey(b_matrix, x_task, x_agent)

    # registo de valores no dataframe
    x_alg   = 'Vickrey Auction'
    x_agent = agent_number
    x_task  = tasks
    df_a = pd.DataFrame([([x_alg, x_agent, x_task, x_total ])], columns=['Algoritmo','Agents', 'Tasks', 'Cost'])

    return df_h, df_a    


#--------------------------------------------------------------------
def main_work(par_number_of_executions, par_var, par_min_cost, par_max_cost, par_max_agents, par_max_tasks):
    
    x_dfh = pd.DataFrame()
    x_dfa = pd.DataFrame()
    dfx   = pd.DataFrame()
    par_i = 1
   # Print date and time     
    now = datetime.datetime.now()
    print (" ")
    print ("Process start at date and time : ", now.strftime("%Y-%m-%d %H:%M:%S"))
    
    while par_i <= par_number_of_executions:   # do n times
        print('========================================================')
        print('=====>> Start Global Cicle :', par_i)
        print('========================================================')
        # Random number for Task and agent
        x_agent = random.randint(2, par_max_agents)   # Minimum 2 Agents
        x_task  = random.randint(2, par_max_tasks)    # Minimum 2 tasks

        bk_agent = x_agent 
        bk_task  = x_task

        print('Task:', x_task,'  Agent:', x_agent)

        # Random tasks and agents Matrix
        k_matrix = np.random.randint(par_min_cost, par_max_cost, size=(x_agent, x_task))
        k_matrix

        # Create matrix wid Bids using original matrix
        #bid_matrix = create_bid_matrix(k_matrix)

        # Global Call
        x_dfh, x_dfa = global_call(x_agent, x_task, k_matrix, par_i)

        # Write results in Pandas DataFrame
        x_dfh.loc[:,'ID'] = par_i
        x_dfa.loc[:,'ID'] = par_i

        hx = x_dfh.iloc[0]['Cost']
        ax = x_dfa.iloc[0]['Cost']
        if hx < ax:                         # Winner Hungarian (Less Cost) 
            x_dfh.loc[:,'Winner_Hungarian'] = 1  
            x_dfh.loc[:,'Winner_Vickrey']   = 0 

            x_dfa.loc[:,'Winner_Hungarian'] = 1  
            x_dfa.loc[:,'Winner_Vickrey']   = 0  
        else:                                # Winner Vickrey (Less Cost) 
            x_dfh.loc[:,'Winner_Hungarian'] = 0 
            x_dfh.loc[:,'Winner_Vickrey']   = 1 

            x_dfa.loc[:,'Winner_Hungarian'] = 0  
            x_dfa.loc[:,'Winner_Vickrey']   = 1          

        dfx = dfx.append(pd.DataFrame(x_dfh),ignore_index=True)
        dfx = dfx.append(pd.DataFrame(x_dfa),ignore_index=True)

        dfx.index = (dfx.index + 1) % len(dfx)
        dfx = dfx.sort_index()
        del x_dfh
        del x_dfa

        par_i += 1

    dfx = dfx.sort_values('ID') 
    
    # Print date and time     
    now = datetime.datetime.now()
    print (" ")
    print ("Process finished at date and time : ", now.strftime("%Y-%m-%d %H:%M:%S"))
   
    # Write results in Excel file 
    print('Finished, look excel file > UC_MR_Mywork.xlsx')
    dfx.to_excel(r'UC_MR_Mywork.xlsx', index = False)
    dfx
    
    return



In [15]:
#-------------------

In [20]:
# http://www.pythonlake.com/tkinterlabelgrid
#--------------------------------------
# Screen to optimizer algorithm Hungarian and Vickrey
#-------------------------------------
from tkinter import messagebox
from tkinter import *
import tkinter as tk

def print_parameters():
    # get values from screen
    p_v0 = float(e0.get())
    p_v1 = float(e1.get())
    p_v2 = float(e2.get())
    p_v3 = float(e3.get())
    p_v4 = float(e4.get())
    p_v5 = float(e5.get())
    #---------------
    print('----  Parameters imputed------')
    print('Number of executions:', p_v0)
    print('Bid price variation :', p_v1)
    print('Min Cost for matrix :', p_v2)
    print('Max Cost for matrix :', p_v3)
    print('Maximum  Agents     :', p_v4)
    print('Maximum  Tasks      :', p_v5)
    print('---------------------------')
  
    main_work(p_v0, p_v1, p_v2, p_v3, p_v4, p_v5 )
    
    message = 'Process done ' +str(p_v0), ' times'
    e_disp = Label(window, text = message,anchor='w').grid(row = 20, column = 1)
    
    return

#------------------------------------------------

message = ''
# Window and title
window = Tk()
window.title("Welcome to Optimizer algorithms app")
window.geometry('800x400')
#-----

# Label
lbl = Label(window, text=" Execution for algorithms Hungarian and Vickrey Auction",font=("Arial Bold", 15))
lbl.grid(row=1, column=1 )

# Label line
lbl = Label(window, text=" __________________________________________________________",font=("Arial Bold", 15))
lbl.grid(row=2, column=1 )
# Label line
lbl = Label(window, text=" ______________",font=("Arial Bold", 15))
lbl.grid(row=2, column=2 )
      
# Button
btn = Button(window, text="Click Me", bg="blue", fg="white", font=("Arial Bold", 10), command=print_parameters)
btn.grid(column=1, row=10)
  
#-- First line 
v0 = IntVar()
v0.set(5)
lbl0 = Label(window, text = "Number of executions:",anchor='w',font=("Arial Bold", 10)).grid(row = 3, column = 1)
e0 = Entry(window , text = v0) 
e0.grid(row = 3, column = 2, sticky="w")

#--
v1 = IntVar()
v1.set(25)
lbl1 = Label(window, text = "Bid price variation :",relief=RIDGE,anchor='w').grid(row = 4, column = 1)
e1 = Entry(window, text = v1) 
e1.grid(row = 4, column = 2, sticky="w")

#--
v2 = IntVar()
v2.set(1)
lbl2 = Label(window, text = "Min Cost for matrix :",anchor='w').grid(row = 5, column = 1)
e2 = Entry(window, text = v2)
e2.grid(row = 5, column = 2, sticky="w")

#--
v3 = IntVar()
v3.set(100)
lbl3 = Label(window, text = "Max Cost for matrix :",anchor='w').grid(row = 6, column = 1)
e3 = tk.Entry(window, text = v3) 
e3.grid(row = 6, column = 2, sticky="w")

#--
v4 = IntVar()
v4.set(10)
lbl4 = Label(window, text = "Maximum  Agents     :",anchor='w').grid(row = 7, column = 1)
e4 = tk.Entry(window, text = v4) 
e4.grid(row = 7, column = 2, sticky="w")

#--
v5 = IntVar()
v5.set(10)
lbl4 = Label(window, text = "Maximum Tasks      :",anchor='w').grid(row = 8, column = 1)
e5 = tk.Entry(window, text = v5) 
e5.grid(row = 8, column = 2, sticky="w")
#--

e_disp = Label(window, text = message,anchor='w').grid(row = 20, column = 1)


window.mainloop()

----  Parameters imputed------
Number of executions: 5.0
Bid price variation : 25.0
Min Cost for matrix : 1.0
Max Cost for matrix : 100.0
Maximum  Agents     : 10.0
Maximum  Tasks      : 10.0
---------------------------
 
Process start at date and time :  2020-12-11 10:06:27
=====>> Start Global Cicle : 1
Task: 6   Agent: 4
============= Hungarian algorithm ( 1 ) ======================
-------------------------------------
|------ Hungarian Algotithm --------|
-------------------------------------
Original Matrix - Agents (row): 4   Tasks (col): 6
Max Value: 99   Min Value: 11
-> Transform to Square Matrix with: 6  by  6
-> Clone Agent: 0  to  4
-> Clone Agent: 1  to  5
Lowest cost through this square  matrix:
[18.0, 63.0, 27.0, 43.0, 49.0, 98.0]
[11.0, 21.0, 82.0, 45.0, 67.0, 92.0]
[81.0, 99.0, 95.0, 20.0, 84.0, 81.0]
[54.0, 48.0, 17.0, 53.0, 33.0, 76.0]
[18.0, 63.0, 27.0, 43.0, 49.0, 98.0]
[11.0, 21.0, 82.0, 45.0, 67.0, 92.0]
--------------
Assigned Tasks Square Matrix
Agent_0 {0}  A

Agent_0 {0}  Assigned task: {1} -> Cost: {27}
Agent_1 {1}  Assigned task: {4} -> Cost: {44}
Agent_2 {2}  Assigned task: {5} -> Cost: {19}
Total cost Non Square: 90
============= Vickrey Auction algorithm ( 5 ) ======================
-------------------------------------
|-- Vickrey Auction Algotithm ------|
-------------------------------------
Original values have Agents: 3    Tasks: 6
    >Original Matrix:
     [[57 27 28 28 29 63]
 [82 70 90 69 44 49]
 [66 66 20 27 91 19]]
After transpose: Rows: 6 Columns: 3   Max: 91   Min: 19
Calc Bid generated winner matrix: [[0 0 0]
 [1 0 0]
 [2 0 0]
 [3 0 0]
 [4 0 0]
 [5 0 0]]
Calc winners - Task number: 0
Calc winners - Task number: 1
Calc winners - Task number: 2
Calc winners - Task number: 3
Calc winners - Task number: 4
Calc winners - Task number: 5
Winners:
-> Start Normalization, with : 6 tasks,  and : 3  Agents
Normalization -> scores : [[3 0 3]]
              -> winners: [[ 0  0 57]
 [ 1  0 27]
 [ 2  2 20]
 [ 3  2 27]
 [ 4  0 29]
 [ 5  

In [12]:
#  ----------------------------------------------------------------

In [19]:
#=======================================
#    Manual Start Process without screen
#=======================================

# --------------  Define parameters to execution
max_value = 999
par_number_of_executions = 20
par_var = 25         # % price variation for bid
par_min_cost = 1     # cost minimum to generate random matrix cost
par_max_cost = 100   # cost minimum to generate random matrix cost
par_max_agents = 50  # maximum agents
par_max_tasks  = 50 # maximum tasks
# ----------------------------------------------
print('----  Run Parameters ------')
print('Number of executions:', par_number_of_executions)
print('Bid price variation :', par_var)
print('Min Cost for matrix :', par_min_cost)
print('Max Cost for matrix :', par_max_cost)
print('Maximum  Agents     :', par_max_agents)
print('Maximum  Tasks      :', par_max_tasks)
print('---------------------------')

#--------------------------

main_work(par_number_of_executions, par_var, par_min_cost, par_max_cost, par_max_agents, par_max_tasks )

#-------------------------------------------

----  Run Parameters ------
Number of executions: 20
Bid price variation : 25
Min Cost for matrix : 1
Max Cost for matrix : 100
Maximum  Agents     : 50
Maximum  Tasks      : 50
---------------------------
 
Process start at date and time :  2020-12-11 10:06:06
=====>> Start Global Cicle : 1
Task: 47   Agent: 32
============= Hungarian algorithm ( 1 ) ======================
-------------------------------------
|------ Hungarian Algotithm --------|
-------------------------------------
Original Matrix - Agents (row): 32   Tasks (col): 47
Max Value: 99   Min Value: 1
-> Transform to Square Matrix with: 47  by  47
-> Clone Agent: 0  to  32
-> Clone Agent: 1  to  33
-> Clone Agent: 2  to  34
-> Clone Agent: 3  to  35
-> Clone Agent: 4  to  36
-> Clone Agent: 5  to  37
-> Clone Agent: 6  to  38
-> Clone Agent: 7  to  39
-> Clone Agent: 8  to  40
-> Clone Agent: 9  to  41
-> Clone Agent: 10  to  42
-> Clone Agent: 11  to  43
-> Clone Agent: 12  to  44
-> Clone Agent: 13  to  45
-> Clone Age

[73, 75, 13, 92, 44, 47, 63, 64, 38, 68, 51, 49, 70, 50, 59, 28, 63, 95, 99, 86, 87, 62, 95, 84, 67, 79, 57, 82, 72, 98,  2, 30, 39, 68, 24, 26, 81, 16, 89, 60, 80, 57, 22, 92, 50, 94, 15]
[40, 98, 20,  6, 80, 82, 12, 23, 95, 21, 55, 44, 68, 49, 10, 85,  6, 10, 13, 79, 50,  9, 81, 86, 29, 60, 64, 61, 32, 53, 73, 25, 66, 42, 96, 37, 62, 84, 72, 42, 31, 44, 16, 27, 77, 27, 21]
[87, 47,  4, 70, 48, 23, 81, 52, 65, 58, 93, 54, 63, 26, 73, 60, 74, 37, 18, 21, 36, 91, 45,  9, 58, 25, 17, 65, 75, 46, 84, 55, 69, 89, 14, 10, 66,  2, 94, 36, 48, 98, 34, 20, 27, 31, 72]
[36, 81, 79, 75, 15, 49, 44, 95, 13, 28, 97, 16, 93, 40, 27, 19, 62, 81, 50, 47, 16, 37, 85, 43, 71, 39, 40, 78, 46, 32, 13, 29, 69, 27, 29, 10, 10, 16, 66, 10, 15, 64, 63, 88, 29,  4, 78]
[74,  5, 44, 56, 59, 91, 71, 54,  9, 47,  7, 67, 18, 62, 90,  4, 28, 62, 98, 69, 61, 30, 96, 83,  6, 27, 49, 17, 52, 24, 58, 76, 18, 83, 35, 25, 82, 44, 71, 13,  2, 85, 51, 68, 82, 65, 92]
[13, 99,  5, 44, 46, 53,  3, 14, 79, 54,  3, 75, 34, 45

  Task >  46  Agent: 22   Bid: 4
Agent scores: [[3 3 4 1 3 3 1 1 1 1 1 1 3 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
--- Calculate totals Vickrey auction----
Bid Total value: 455
Winners Matrix row=Tasks , Col_0 =Task number, Col_1=Agent, Col_2=Bid Value
=====>> Start Global Cicle : 2
Task: 44   Agent: 31
============= Hungarian algorithm ( 2 ) ======================
-------------------------------------
|------ Hungarian Algotithm --------|
-------------------------------------
Original Matrix - Agents (row): 31   Tasks (col): 44
Max Value: 99   Min Value: 1
-> Transform to Square Matrix with: 44  by  44
-> Clone Agent: 0  to  31
-> Clone Agent: 1  to  32
-> Clone Agent: 2  to  33
-> Clone Agent: 3  to  34
-> Clone Agent: 4  to  35
-> Clone Agent: 5  to  36
-> Clone Agent: 6  to  37
-> Clone Agent: 7  to  38
-> Clone Agent: 8  to  39
-> Clone Agent: 9  to  40
-> Clone Agent: 10  to  41
-> Clone Agent: 11  to  42
-> Clone Agent: 12  to  43
Lowest cost through this square  matrix:
[16.0, 

[28.0, 72.0, 39.0, 82.0,  2.0, 36.0, 21.0, 51.0,  4.0, 45.0, 91.0,  1.0, 11.0,  3.0, 91.0, 55.0, 55.0, 79.0, 74.0, 99.0, 75.0, 87.0, 94.0,  4.0, 14.0, 38.0, 89.0,  9.0, 94.0,  7.0, 34.0,  9.0, 80.0, 79.0, 52.0, 69.0, 54.0, 91.0, 38.0, 30.0,  3.0, 99.0, 31.0, 72.0]
[85.0, 40.0, 74.0, 35.0, 93.0, 42.0,  4.0, 66.0, 83.0, 82.0, 24.0, 22.0, 33.0, 57.0, 83.0, 78.0, 51.0,  5.0, 40.0, 56.0, 37.0, 12.0, 41.0, 68.0, 39.0, 78.0, 33.0, 46.0,  7.0, 44.0, 48.0, 87.0, 94.0,  9.0,  4.0, 47.0, 35.0, 69.0, 88.0,  9.0, 69.0, 75.0, 51.0, 90.0]
[10.0, 47.0,  7.0, 63.0, 14.0, 50.0, 64.0, 48.0, 33.0, 84.0, 36.0, 14.0,  5.0, 57.0, 74.0, 92.0, 21.0, 38.0, 79.0, 75.0, 93.0,  1.0,  7.0, 59.0, 25.0, 97.0, 36.0, 78.0, 39.0, 51.0, 14.0, 92.0, 98.0, 47.0, 18.0, 15.0, 24.0, 19.0, 21.0,  5.0, 31.0, 65.0, 74.0, 86.0]
[11.0, 11.0, 56.0, 77.0, 81.0, 46.0, 66.0, 68.0, 20.0, 21.0, 27.0, 85.0, 64.0, 24.0, 96.0, 47.0, 37.0, 30.0, 54.0, 84.0, 19.0, 86.0,  4.0, 65.0, 25.0,  5.0, 67.0, 89.0, 45.0,  7.0, 30.0, 30.0, 46.0, 34.0, 

Agent_12 {12}  Assigned task: {0} -> Cost: {6}
Agent_13 {13}  Assigned task: {20} -> Cost: {2}
Agent_14 {14}  Assigned task: {19} -> Cost: {1}
Agent_15 {15}  Assigned task: {9} -> Cost: {4}
Agent_16 {16}  Assigned task: {23} -> Cost: {6}
Agent_17 {17}  Assigned task: {32} -> Cost: {3}
Agent_18 {18}  Assigned task: {25} -> Cost: {1}
Agent_19 {19}  Assigned task: {35} -> Cost: {1}
Agent_20 {20}  Assigned task: {15} -> Cost: {6}
Agent_21 {21}  Assigned task: {31} -> Cost: {3}
Agent_22 {22}  Assigned task: {3} -> Cost: {4}
Agent_23 {23}  Assigned task: {29} -> Cost: {9}
Agent_24 {24}  Assigned task: {8} -> Cost: {2}
Agent_25 {25}  Assigned task: {43} -> Cost: {4}
Agent_26 {26}  Assigned task: {28} -> Cost: {2}
Agent_27 {27}  Assigned task: {27} -> Cost: {3}
Agent_28 {28}  Assigned task: {16} -> Cost: {7}
Agent_29 {29}  Assigned task: {18} -> Cost: {2}
Agent_30 {30}  Assigned task: {26} -> Cost: {1}
Total cost Non Square: 104
============= Vickrey Auction algorithm ( 2 ) ===================

...

    Tie agent: 28 Task: 5 Max Bid: 5  Scores: [[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 1 0 0 0 1 0 1 0 0 0 0 0]]
    ---> Best bid tie, first bid, agent -> 28   Bid: 5
Winners:
-> Start Normalization, with : 6 tasks,  and : 50  Agents
Normalization -> scores : [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 1 0 0 0 1 0 1 0 0 0 0 0]]
              -> winners: [[ 0 42  1]
 [ 1 21  5]
 [ 2 29  2]
 [ 3 44  4]
 [ 4 38  4]
 [ 5 35  4]]
-> Finish Normalization
Initial Create scores_1: 50 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 1 0 0 0 1 0 1 0 0 0 0 0]]
[[ 0  1  2  3  4  5]
 [42 21 29 44 38 35]
 [ 1  5  2  4  4  4]]
-------------------------
WINNERS :
  Task >  0  Agent: 42   Bid: 1
  Task >  1  Agent: 21   Bid: 5
  Task >  2  Agent: 29   Bid: 2
  Task >  3  Agent: 44   Bid: 4
  Task >  4  Agent: 38   Bid: 4
  Task >  5  Agent: 35   Bid: 4
Agent scores: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

=====>> Start Global Cicle : 14
Task: 5   Agent: 28
============= Hungarian algorithm ( 14 ) ======================
-------------------------------------
|------ Hungarian Algotithm --------|
-------------------------------------
Original Matrix - Agents (row): 28   Tasks (col): 5
Max Value: 99   Min Value: 2
Lowest cost through this square  matrix:
[ 9, 76, 62, 98, 16]
[63, 57, 85, 78,  6]
[90, 34, 90, 10, 87]
[42, 17, 82, 16, 92]
[44, 47, 33, 57, 68]
[90, 82, 30, 22, 73]
[10, 15, 48, 72, 62]
[68, 99, 70, 32,  9]
[43, 24,  3,  5, 39]
[88, 44, 10, 18, 43]
[21, 27, 76, 90, 85]
[26, 23, 63, 56, 28]
[49, 67,  2, 94, 71]
[75, 14, 99, 79,  6]
[51, 68, 25,  3, 87]
[24, 82, 78, 99, 27]
[44, 75, 28, 82, 16]
[82, 20, 28, 52,  6]
[25, 89, 81, 82, 52]
[27, 40, 74, 25, 62]
[81, 15, 51, 56, 57]
[49, 52,  7, 31, 39]
[57, 82,  7, 28,  7]
[66, 19, 96, 22, 18]
[97, 51, 77, 16, 65]
[ 3, 56, 32, 17, 28]
[63, 13, 53,  2, 36]
[47, 71,  6, 44, 13]
--------------
Assigned Tasks Square Matrix
Agent_1 {1}  Ass

Lowest cost through this non square matrix:
[78, 33, 30, 37, 21, 34,  7, 57, 28, 70, 20,  7, 69, 90, 27, 63, 83]
[76, 73, 64, 29, 84, 35, 62, 38, 62, 47, 30, 31, 41, 94, 24, 88, 22]
[68, 98, 96, 98, 92, 16, 49, 20, 75, 50, 33, 67, 87, 62, 46, 35, 67]
[37, 70,  4, 52, 73, 18, 45, 44,  6, 34, 37, 21, 62, 77, 48,  6, 51]
[70, 18, 83, 97, 75, 19, 91, 34, 93, 68, 95, 25, 91, 80, 43, 98, 82]
[68, 59, 25, 37, 78, 43, 48, 90,  2, 72, 11, 15, 46, 51, 12,  2, 95]
[95,  6, 22, 89, 16, 53, 22, 28, 21,  4, 93, 71, 13,  6, 13, 54, 93]
[20, 95, 43, 89,  2, 76, 36, 99,  1, 99, 65,  5, 62, 48, 86, 33, 96]
[29, 13, 16, 75, 88, 62, 64, 66, 36, 36, 53, 30, 33, 18, 80, 50, 23]
[98, 62, 80, 50, 49, 17, 86, 91, 95, 61, 22, 31, 25, 81, 83, 70,  7]
--------------
Assigned Tasks Non Square Matrix
Agent_0 {0}  Assigned task: {6} -> Cost: {7}
Agent_1 {1}  Assigned task: {14} -> Cost: {24}
Agent_2 {2}  Assigned task: {5} -> Cost: {16}
Agent_3 {3}  Assigned task: {2} -> Cost: {4}
Agent_4 {4}  Assigned task: {1} -> 

[77, 98, 54, 79, 37, 55, 15, 56, 70,  6, 39, 84, 36, 74, 77, 66, 85]
[27, 22,  6, 30, 99, 68, 32, 52, 65, 42,  9, 39, 68, 35,  8, 95, 45]
[69, 88, 72, 14,  2, 45, 70, 98,  1, 10, 18,  8, 63, 54, 44, 16,  7]
[17, 86,  6, 75, 70, 65, 13, 91, 44, 59, 82, 65, 74, 16, 26, 38, 16]
[24, 30, 32,  5, 95, 76, 65, 36, 59, 47, 43, 38, 46, 11, 41, 12, 61]
[57, 22, 47, 53, 31, 70,  3, 96, 71, 62, 29, 99, 54, 42, 57, 78, 24]
[48, 65, 68, 14, 76, 99, 22, 60,  1, 23, 94,  4, 52, 27, 35, 40, 48]
[43, 75, 56, 91, 48, 52,  4, 29,  6,  7, 32, 15, 33, 77, 12, 35, 79]
--------------
Assigned Tasks Non Square Matrix
Agent_0 {0}  Assigned task: {15} -> Cost: {3}
Agent_1 {1}  Assigned task: {10} -> Cost: {4}
Agent_2 {2}  Assigned task: {12} -> Cost: {9}
Agent_10 {10}  Assigned task: {14} -> Cost: {2}
Agent_12 {12}  Assigned task: {5} -> Cost: {4}
Agent_13 {13}  Assigned task: {16} -> Cost: {5}
Agent_21 {21}  Assigned task: {11} -> Cost: {1}
Agent_22 {22}  Assigned task: {9} -> Cost: {3}
Agent_23 {23}  Assigned 

=====>> Start Global Cicle : 19
Task: 10   Agent: 8
============= Hungarian algorithm ( 19 ) ======================
-------------------------------------
|------ Hungarian Algotithm --------|
-------------------------------------
Original Matrix - Agents (row): 8   Tasks (col): 10
Max Value: 99   Min Value: 1
-> Transform to Square Matrix with: 10  by  10
-> Clone Agent: 0  to  8
-> Clone Agent: 1  to  9
Lowest cost through this square  matrix:
[32.0, 80.0, 68.0, 61.0, 88.0, 33.0, 33.0, 79.0, 64.0, 26.0]
[54.0, 38.0,  6.0, 68.0, 43.0, 32.0, 70.0, 69.0, 46.0,  5.0]
[25.0, 40.0, 40.0, 25.0, 52.0, 75.0,  5.0, 29.0, 55.0, 69.0]
[45.0, 28.0, 10.0, 17.0, 92.0,  4.0, 45.0, 14.0, 23.0,  8.0]
[27.0, 18.0, 70.0, 73.0,  4.0, 66.0, 88.0, 17.0, 33.0, 18.0]
[74.0, 23.0, 51.0, 17.0, 42.0,  1.0, 11.0, 96.0, 86.0,  6.0]
[48.0, 73.0, 41.0, 89.0, 96.0, 99.0, 24.0, 62.0, 14.0, 39.0]
[35.0, 85.0, 34.0, 53.0, 16.0, 53.0, 15.0, 73.0, 25.0, 92.0]
[32.0, 80.0, 68.0, 61.0, 88.0, 33.0, 33.0, 79.0, 64.0, 26.0]
[5

In [105]:
'''
# ---- Block only for TEST ----
#
# Rows - Agents
# Columns = tasks
import numpy as np

matrix = []
matrix = [[1, 9, 6, 4, 1, 9],
[8, 4, 6, 4, 4, 5],
[7, 9, 9, 5, 3, 4],
[2, 4, 5, 1, 7, 6],
[3, 5, 5, 3, 3, 2]]

winners =[[ 0 , 1 ,21],
          [ 1 , 0 ,1],
          [ 2 , 0 ,1],
          [ 3 , 0 ,1],        
          [ 4 , 0 ,1]]
# Transform list to numpy array
matrix_arr = np.array(matrix)

bk_agent = 5
bk_task  = 6

#create_scores(winners_arr, x_agents, bk_task, 1)

#------- TEST AUCTION ----------------------
l=0    
max_value = max([max(l) for l in matrix_arr])
min_value = min([min(l) for l in matrix_arr])
print('Max:',max_value , '   Min:', min_value)
    
x_total = main_auction_vickrey(matrix_arr, bk_task, bk_agent)

#'''
'''
#----------------------------------
# TEST Normalization - Some agents with many task and others with none
#matrix_t = matrix_arr.transpose()
print('TESTE NORMALIZATION')    
score =  [[3, 1, 0, 0, 1]]
score_arr = np.array(score)
print('Scores:',score_arr)

winners =[[ 0 , 4 ,21],
          [ 6 , 0 ,1]]
winners_arr = np.array(winners)
print('Winners:')
winners_arr

# Transpose
matrix_arr = np.array(matrix)
matrix_t = matrix_arr.transpose()
print('Matrix_transposed:')
matrix_t

winners_arr, score_arr  = verify_normalization(matrix_t, score_arr, winners_arr)
#----------------------------------------

print('--- agents -----')
#print(x_matrix_t)
print('----------------')
'''

"\n#----------------------------------\n# TEST Normalization - Some agents with many task and others with none\n#matrix_t = matrix_arr.transpose()\nprint('TESTE NORMALIZATION')    \nscore =  [[3, 1, 0, 0, 1]]\nscore_arr = np.array(score)\nprint('Scores:',score_arr)\n\nwinners =[[ 0 , 4 ,21],\n          [ 6 , 0 ,1]]\nwinners_arr = np.array(winners)\nprint('Winners:')\nwinners_arr\n\n# Transpose\nmatrix_arr = np.array(matrix)\nmatrix_t = matrix_arr.transpose()\nprint('Matrix_transposed:')\nmatrix_t\n\nwinners_arr, score_arr  = verify_normalization(matrix_t, score_arr, winners_arr)\n#----------------------------------------\n\nprint('--- agents -----')\n#print(x_matrix_t)\nprint('----------------')\n"